In [2]:
import numpy as np
import csv

def floatize(the_list):
    return map(lambda x:float(x),the_list)

def load_world_points():
    world_reader=csv.reader(open("Files/world.csv"),delimiter=' ')
    return np.array(map(floatize,list(world_reader)))

def load_image_points():
    image_reader=csv.reader(open("Files/image.csv"),delimiter=' ')
    return np.array(map(floatize,list(image_reader)))

world_points=load_world_points()
image_points=load_image_points()
print(world_points)
print(image_points)

[[ 0.8518447   0.75947939  0.94975928]
 [ 0.55793851  0.01423302  0.59617708]
 [ 0.81620571  0.97709235  0.22190808]
 [ 0.70368367  0.52206092  0.93289706]
 [ 0.71335444  0.2280389   0.4496421 ]
 [ 0.1721997   0.96882014  0.3557161 ]
 [ 0.04904683  0.75533857  0.89481276]
 [ 0.28614965  0.25120055  0.93273619]
 [ 0.13098247  0.94081954  0.70185317]
 [ 0.84767647  0.20927164  0.45509169]]
[[  5.11770701   4.76538441]
 [  5.5236545    3.87032918]
 [  7.16310171   7.35942066]
 [  5.22216628   4.4279585 ]
 [  5.60479614   4.67483648]
 [ 13.59494885  10.05215495]
 [  8.73452189   5.56420531]
 [  6.22433952   3.90821885]
 [  9.74763886   6.90423723]
 [  5.09031079   4.55085131]]


In [3]:
def make_homogenous(le_vector):
    le_shape=le_vector.shape
    ones=np.transpose(np.atleast_2d(np.ones(le_shape[0])))
    return np.concatenate((le_vector,ones),axis=1)
h_world_pts=make_homogenous(world_points)
h_image_pts=make_homogenous(image_points)
print(h_world_pts)
print(h_image_pts)

[[ 0.8518447   0.75947939  0.94975928  1.        ]
 [ 0.55793851  0.01423302  0.59617708  1.        ]
 [ 0.81620571  0.97709235  0.22190808  1.        ]
 [ 0.70368367  0.52206092  0.93289706  1.        ]
 [ 0.71335444  0.2280389   0.4496421   1.        ]
 [ 0.1721997   0.96882014  0.3557161   1.        ]
 [ 0.04904683  0.75533857  0.89481276  1.        ]
 [ 0.28614965  0.25120055  0.93273619  1.        ]
 [ 0.13098247  0.94081954  0.70185317  1.        ]
 [ 0.84767647  0.20927164  0.45509169  1.        ]]
[[  5.11770701   4.76538441   1.        ]
 [  5.5236545    3.87032918   1.        ]
 [  7.16310171   7.35942066   1.        ]
 [  5.22216628   4.4279585    1.        ]
 [  5.60479614   4.67483648   1.        ]
 [ 13.59494885  10.05215495   1.        ]
 [  8.73452189   5.56420531   1.        ]
 [  6.22433952   3.90821885   1.        ]
 [  9.74763886   6.90423723   1.        ]
 [  5.09031079   4.55085131   1.        ]]


In [36]:
def construct_A_row(row_num,Is,Ws):
    n=row_num/2
    shift=row_num%2
    Wn=Ws[n]
    In=Is[n]
    if shift==0:
        return [Wn[0],Wn[1],Wn[2],Wn[3],0,0,0,0,-In[0]*Wn[0],-In[0]*Wn[1],-In[0]*Wn[2],-In[0]]
    elif shift==1:
        return [0,0,0,0,Wn[0],Wn[1],Wn[2],Wn[3],-In[1]*Wn[0],-In[1]*Wn[1],-In[1]*Wn[2],-In[1]]
    
def construct_A(Is,Ws):
    A=[]
    for row_num in range(2*Ws.shape[0]):
        A.append(construct_A_row(row_num,Is,Ws))    
    return np.array(A)

def construct_P(Is,Ws):
    A=construct_A(Is,Ws)
    u,s,v=np.linalg.svd(A)
    P=np.reshape(v[-1],(3,4))
#     print(np.dot(A,v[-1]))
    return P

P=construct_P(h_image_pts,h_world_pts)
# print(np.dot(P,h_world_pts))
print(np.transpose(np.dot(P,np.transpose(h_world_pts)))*4.370554223196884)

[[ 5.11770702  4.76538442  1.        ]
 [ 3.53847786  2.47935024  0.64060449]
 [ 4.1274968   4.240619    0.57621642]
 [ 4.74382649  4.02236652  0.90840204]
 [ 3.61808624  3.0177657   0.64553396]
 [ 3.98366588  2.94553714  0.29302544]
 [ 4.57601247  2.9150849   0.52389959]
 [ 4.21111377  2.64412861  0.67655592]
 [ 4.50608523  3.19165306  0.46227454]
 [ 3.68088376  3.29079212  0.72311572]]


In [31]:
 4.76538441/1.0903387

4.370554223196884